# Modularity
Implements formula's from Lambiotte et al., 2008.

In [1]:
import numpy as np
import scipy.sparse

In [2]:
# Create adjacency matrix
n = 6
A = np.zeros((n, n))
A[0,1] = 2
A[0,2] = 2
A[1,2] = 2
A[2,3] = 1
A[3,5] = 2
A[3,4] = 2
A[4,5] = 2

A = A + np.triu(A).T
m = A.sum() / 2
A

array([[ 0.,  2.,  2.,  0.,  0.,  0.],
       [ 2.,  0.,  2.,  0.,  0.,  0.],
       [ 2.,  2.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  2.,  2.],
       [ 0.,  0.,  0.,  2.,  0.,  2.],
       [ 0.,  0.,  0.,  2.,  2.,  0.]])

In [3]:
# assign to commnities
ncom = 2
C = np.zeros((n, ncom))
for i in range(3):
    C[i, 0] = 1
for j in range(3, 6):
    C[j, 1] = 1
C

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [4]:
def get_modularity_matrix(mat):
    # k_i is the degree of k
    k = mat.sum(axis=0)
    mod = mat - (np.outer(k, k) / np.sum(mat))
    return mod

In [5]:
def get_modularity_value(mat, partition):
    # delta tells if i,j are in the same cluster
    delta = partition.dot(partition.T)
    mod = get_modularity_matrix(mat)
    Q = 1.0 / np.sum(mat) * np.sum(delta*mod)
    return Q

In [6]:
def get_init_stability(mat, partition):
    k = mat.sum(axis=0)
    delta = partition.dot(partition.T)
    return 1 - np.sum(delta*np.outer(k, k)/ np.square(sum(mat)))

In [7]:
def get_approx_stability(mat, partition, t):
    stab_init = get_init_stability(mat, partition)
    modularity_value = get_modularity_value(mat, partition)
    return (1.0-t)*stab_init + t*modularity_value

In [8]:
Q = get_modularity_value(A, C)
Q

0.42307692307692313

In [9]:
get_modularity_value(scipy.sparse.csr_matrix(A), C)

0.0

In [10]:
B = get_modularity_matrix(A)
B

array([[-0.61538462,  1.38461538,  1.23076923, -0.76923077, -0.61538462,
        -0.61538462],
       [ 1.38461538, -0.61538462,  1.23076923, -0.76923077, -0.61538462,
        -0.61538462],
       [ 1.23076923,  1.23076923, -0.96153846,  0.03846154, -0.76923077,
        -0.76923077],
       [-0.76923077, -0.76923077,  0.03846154, -0.96153846,  1.23076923,
         1.23076923],
       [-0.61538462, -0.61538462, -0.76923077,  1.23076923, -0.61538462,
         1.38461538],
       [-0.61538462, -0.61538462, -0.76923077,  1.23076923,  1.38461538,
        -0.61538462]])

In [11]:
get_init_stability(A, C)

-17.199999999999999

In [12]:
get_approx_stability(A, C, 0.5)

-8.388461538461538

## compare with reference implementation

In [13]:
import networkx as nx
import community

In [14]:
graph = nx.Graph(A)
nz = C.nonzero()
partition = {nz[0][i]: nz[1][i] for i in range(len(nz[0]))}
community.modularity(partition, graph)

0.42307692307692313

In [15]:
import igraph
import louvain

In [16]:
g = igraph.Graph()
g.add_vertices(len(A))
ix, jx = A.nonzero()
for i,j in zip(ix, jx):
    if i<j:
        g.add_edge(i, j, weight=A[i,j])

In [17]:
part = louvain.find_partition(g, louvain.ModularityVertexPartition, weights='weight')
print(part.membership)
print(part.modularity)
print(part.quality())

[0, 0, 0, 1, 1, 1]
0.3571428571428571
0.4230769230769231


In [94]:
B = np.array([[ 0.,  2.,  2.,  0.,  0.,  0.],
       [ 2.,  0.,  2.,  0.,  0.,  0.],
       [ 2.,  2.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  2.,  2.],
       [ 0.,  0.,  0.,  2.,  0.,  2.],
       [ 0.,  0.,  0.,  2.,  2.,  0.]])

In [96]:
A==B

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]], dtype=bool)

In [68]:
part.membership

[0, 0, 0, 1, 1, 1]

In [69]:
g.write_graphml('/media/sf_VBox_Shared/networks/test.graphml')

Gephi notes a modularity of 0.423, without weight itś 0.357

In [70]:
part = louvain.find_partition(g, louvain.ModularityVertexPartition)
part.modularity

0.3571428571428571

In [71]:
part.quality()

0.35714285714285715

In [72]:
g.es['weight']

[2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0]

In [87]:
w_tot = 13*2
((2-4*4/w_tot)*4 + (2-4*5/w_tot)*8 + (1-5*5/w_tot)*2 + (0-5*5/w_tot)*2)/w_tot

0.5207100591715976

In [84]:
delta = C.dot(C.T)
get_modularity_matrix(A)*13*2 * delta

array([[-16.,  36.,  32.,  -0.,  -0.,  -0.],
       [ 36., -16.,  32.,  -0.,  -0.,  -0.],
       [ 32.,  32., -25.,   0.,  -0.,  -0.],
       [ -0.,  -0.,   0., -25.,  32.,  32.],
       [ -0.,  -0.,  -0.,  32., -16.,  36.],
       [ -0.,  -0.,  -0.,  32.,  36., -16.]])

In [86]:
get_modularity_matrix(A)

array([[-0.61538462,  1.38461538,  1.23076923, -0.76923077, -0.61538462,
        -0.61538462],
       [ 1.38461538, -0.61538462,  1.23076923, -0.76923077, -0.61538462,
        -0.61538462],
       [ 1.23076923,  1.23076923, -0.96153846,  0.03846154, -0.76923077,
        -0.76923077],
       [-0.76923077, -0.76923077,  0.03846154, -0.96153846,  1.23076923,
         1.23076923],
       [-0.61538462, -0.61538462, -0.76923077,  1.23076923, -0.61538462,
         1.38461538],
       [-0.61538462, -0.61538462, -0.76923077,  1.23076923,  1.38461538,
        -0.61538462]])

In [92]:
louvain.find_partition?

Signature: louvain.find_partition(graph, partition_type, initial_membership=None, weights=None, **kwargs)
Docstring:
Detect communities using the default settings.

This function detects communities given the specified method in the
``partition_type``. This should be type derived from
:class:`VertexPartition.MutableVertexPartition`, e.g.
:class:`ModularityVertexPartition` or :class:`CPMVertexPartition`. Optionally
an initial membership and edge weights can be provided. Remaining
``**kwargs`` are passed on to the constructor of the ``partition_type``,
including for example a ``resolution_parameter``.

Parameters
----------
graph : :class:`ig.Graph`
  The graph for which to detect communities.

partition_type : type of :class:`
  The type of partition to use for optimisation.

initial_membership : list of int
  Initial membership for the partition. If :obj:`None` then defaults to a
  singleton partition.

weights : list of double, or edge attribute
  Weights of edges. Can be either an it